**NOTE**: Here we test the performance of the STged with true cell type proportion and cell type proportion predicted from cell type deconvolution methods. We then compare it with competing methods.

In [1]:

#remove(list = ls())
#define the conda env

library(reticulate)
use_condaenv("ccnet", required = TRUE)

python_env = "python_env"

In [2]:
library(ggplot2)
library(scatterpie)
library(RColorBrewer)
library(TCA)
library(ENIGMA)

Package 'ENIGMA' version 0.1.6
ENIGMA is built for fully deconvolution.



In [3]:
# Source code
source("./code/STged.R")
source("./code/benchmark.R")

In [4]:
colors <- colorRampPalette(brewer.pal(9, "Set3"))(12)


# Load data the generated simulated data



In [5]:
sim_data = readRDS("./realdata/MPOA/FN7_hash_mpoa_sim-29_ref01.Rds")

i = 2
patch_size = c(100,50)[i]
patch_size

mpoa = sim_data$mpoa_list[[i]]$`-0.29`

# load the referene from scRNA
sim_data = readRDS("./realdata/MPOA/refscRNA/scRNA_MPOA50k.RDS")
sc_exp = sim_data$scexp
sc_label = sim_data$sclabel

dim(sc_exp)
length(sc_label)


[1] 50

Loading required package: hash



hash-2.2.6.3 provided by Decision Patterns




[1]   134 29760

[1] 29760

In [6]:

table(sc_label)



sc_label
  Astrocyte Endothelial   Ependymal  Excitatory  Inhibitory   Microglia 
        857         541          69        3511       15042         828 
OD Immature   OD Mature   Pericytes 
       1692        6707         513 

## The spots location information

In [7]:
spot_raw = rownames(mpoa$patchGexp)
spots = paste0("spot",1:length(spot_raw))


split_strings <- strsplit(spot_raw, "_")
spot_loc <- do.call(rbind, lapply(split_strings, function(x) as.numeric(x)))
spot_exp <- t(mpoa$patchGexp)[,spot_raw ]
dim(spot_exp)

cell_type_freq = as.matrix(mpoa$cellTypeTable[spot_raw,]) 
cell_type_freq <- apply(cell_type_freq, 2, as.numeric)  
cell_type_prop = sweep(cell_type_freq, 1, rowSums(cell_type_freq), FUN = "/")

cell_type = colnames(cell_type_prop)
cell_type


rownames(cell_type_prop) = rownames(spot_loc)  = spots
colnames(spot_exp) = spots


[1]  135 1141

[1] "Astrocyte"   "Endothelial" "Ependymal"   "Excitatory"  "Inhibitory" 
[6] "Microglia"   "OD Immature" "OD Mature"   "Pericytes"

## TRUE cell type-specific gene expression for each cell type

In [8]:
genes = rownames(spot_exp)  
TRUE_F = vector("list", length(cell_type))  
names(TRUE_F) = cell_type  

spot_cell_type_exp = mpoa$avgPatchGexp

for(i in 1:length(cell_type)){
  
  temp_mat = matrix(0, nrow = length(spots), ncol = length(genes))
  colnames(temp_mat) = genes
  rownames(temp_mat) = spots
  
  for(j in 1:length(spots)){
    temp = spot_cell_type_exp[[j]]
    
    
    if (cell_type[i] %in% colnames(temp)) {
      temp_mat[j,] = temp[,cell_type[i]]
    } else {
      
      temp_mat[j,] = 0
    }
  }
  
  
  TRUE_F[[i]] = t(temp_mat)
}



# Run the STged step by step

## Step 1: clear data

In [9]:
clean.only = FALSE
depthscale = 1e6
datax = data_process(sc_exp = sc_exp,   sc_label = sc_label, 
                     spot_exp = spot_exp,  spot_loc = spot_loc,
                     depthscale = depthscale,  gene_det_in_min_cells_per = 0, 
                     expression_threshold = 0,
                     nUMI = 10, verbose = FALSE, clean.only = clean.only)


In [10]:
max(datax$spot_exp)
max(datax$sc_exp)

[1] 13.30836

[1] 13.75724

In [11]:
dim(datax$sc_exp)
table(datax$sc_label)

[1]   134 28357


  Astrocyte Endothelial   Ependymal  Excitatory  Inhibitory   Microglia 
        574         514          54        3418       14304         730 
OD Immature   OD Mature   Pericytes 
       1647        6608         508 

In [12]:
# use for the other completed methods
datax_count = data_process(sc_exp = sc_exp,   sc_label = sc_label, 
                     spot_exp = spot_exp,  spot_loc = spot_loc,
                     depthscale = depthscale,  gene_det_in_min_cells_per = 0, 
                     expression_threshold = 0,
                     nUMI =  10, verbose = FALSE, clean.only = TRUE)

In [13]:
genes_sec = rownames(datax$spot_exp)
length(genes_sec)

[1] 134

In [14]:
genes_sec = rownames(datax$spot_exp)

for(i in 1:length(cell_type)){
  
  TRUE_F[[i]] = TRUE_F[[i]][genes_sec, colnames(datax$spot_exp )]
  
}

depthscale = 1e6
trueF = res_library_size_normaization(estres = TRUE_F, depthscale = depthscale)


saveRDS(TRUE_F, file = paste0("./results/MPOA/True_cell_exp_", patch_size, "um2.RDS"))


## Step 2: construct spatial correlation structures

In [15]:
cat("Construct spatial correlation", "\n")
L.mat = dis_weight(spot_loc = datax$spot_loc, spot_exp = datax$spot_exp, k = 4, 
                   quantile_prob_bandwidth = 1/3, method = "Square", 
                   coord_type = "grid")

Construct spatial correlation 


## Step 3: construct reference gene matrix

In [16]:
cat("Construct reference gene matrix", "\n")

ref_exp = create_group_exp(sc_exp = datax$sc_exp, sc_label = datax$sc_label)
colnames(ref_exp)


Construct reference gene matrix 


[1] "Astrocyte"   "Endothelial" "Ependymal"   "Excitatory"  "Inhibitory" 
[6] "Microglia"   "OD Immature" "OD Mature"   "Pericytes"

In [17]:
ref_exp = ref_exp[rownames(datax$spot_exp),]
colnames(ref_exp )
cell_type

[1] "Astrocyte"   "Endothelial" "Ependymal"   "Excitatory"  "Inhibitory" 
[6] "Microglia"   "OD Immature" "OD Mature"   "Pericytes"

[1] "Astrocyte"   "Endothelial" "Ependymal"   "Excitatory"  "Inhibitory" 
[6] "Microglia"   "OD Immature" "OD Mature"   "Pericytes"

In [18]:
beta.type = cell_type_prop[colnames(datax$spot_exp),]

In [19]:
p = nrow(datax$spot_exp)
p
K = ncol(beta.type)
beta.ind.temp = beta.type>0.05

beta.ind =  matrix(list(), K ,1)
for (i in 1:K){
  
  beta.ind[[i]] =  matrix( rep(beta.ind.temp[,i],p), nrow = p, byrow = TRUE)
}

saveRDS(beta.ind, file =paste0("./results/MPOA/cell_prop_indx_",patch_size,"um2.RDS"))


[1] 134

## Step 4: run the main model
### Step 4-1: run the main model with true cell type proportion

In [20]:
 
start_time <- Sys.time()

if (!file.exists(paste0("./results/MPOA/True_stged_",patch_size,"um2.RDS"))) {
  stged.est.true <- MUR.STged(
    srt_exp = datax$spot_exp, 
    ref_exp = ref_exp, 
    beta.type = beta.type, 
    W = L.mat$weight_adj, 
    lambda1 = NULL, 
    lambda2 = NULL, 
    cutoff = 0.05, 
    epsilon = 1e-5
  )
  
  saveRDS(stged.est.true, file = paste0( "./results/MPOA/True_stged_",patch_size,"um2.RDS"))
} else {
  
  stged.est.true <- readRDS(paste0("./results/MPOA/True_stged_",patch_size,"um2.RDS"))
}

end_time <- Sys.time()


reg1 <- TRUE

if (reg1 && !file.exists(paste0("./results/MPOA/True_stged_reg1_",patch_size,"um2.RDS"))) {
  stged.est.reg1 <- MUR.STged(
    srt_exp = datax$spot_exp, 
    ref_exp = ref_exp, 
    beta.type = beta.type, 
    W = L.mat$weight_adj, 
    lambda1 = NULL, 
    lambda2 = 0, 
    cutoff = 0.05, 
    epsilon = 1e-5
  )
  
  saveRDS(stged.est.reg1, file = paste0("./results/MPOA/True_stged_reg1_",patch_size,"um2.RDS"))
} else {
  
  stged.est.reg1 <- readRDS(paste0("./results/MPOA/True_stged_reg1_",patch_size,"um2.RDS"))
}


reg2 <- TRUE
if (reg2 && !file.exists(paste0("./results/MPOA/True_stged_reg2_",patch_size,"um2.RDS"))) {
  stged.est.reg2 <- MUR.STged(
    srt_exp = datax$spot_exp, 
    ref_exp = ref_exp, 
    beta.type = beta.type, 
    W = L.mat$weight_adj, 
    lambda1 = 0, 
    lambda2 = NULL, 
    cutoff = 0.05, 
    epsilon = 1e-5
  )
  
  saveRDS(stged.est.reg2, file = paste0("./results/MPOA/True_stged_reg2_",patch_size,"um2.RDS"))
} else {
  
  stged.est.reg2 <- readRDS(paste0("./results/MPOA/True_stged_reg2_",patch_size,"um2.RDS"))
}



reg3 <- TRUE
if (reg3 && !file.exists(paste0("./results/MPOA/True_stged_reg3_",patch_size,"um2.RDS"))) {
  stged.est.reg3 <- MUR.STged(
    srt_exp = datax$spot_exp, 
    ref_exp = ref_exp, 
    beta.type = beta.type, 
    W = L.mat$weight_adj, 
    lambda1 = 0, 
    lambda2 = 0, 
    cutoff = 0.05, 
    epsilon = 1e-5
  )
  
  saveRDS(stged.est.reg3, file = paste0("./results/MPOA/True_stged_reg3_",patch_size,"um2.RDS"))
} else {
  
  stged.est.reg3 <- readRDS(paste0("./results/MPOA/True_stged_reg3_",patch_size,"um2.RDS"))
}

We will adpote a value for lambda 1 in our algorithm... 
Select value of lambda1 0.08262865 
tuning for lambda 2 in our algorithm... 
Select value of lambda2 0.1951821 
Run the main algorithm... 


We will adpote a value for lambda 1 in our algorithm... 
Select value of lambda1 0.08262865 
Select value of lambda2 0 
Run the main algorithm... 


Select value of lambda1 0 
tuning for lambda 2 in our algorithm... 
Select value of lambda2 0.1951821 
Run the main algorithm... 


Select value of lambda1 0 
Select value of lambda2 0 
Run the main algorithm... 


In [21]:
# the optial values                      
lambda1_opt = stged.est.true$lambda1
lambda2_opt = stged.est.true$lambda2

# Run the benchmarking methods

In [22]:
exp_lsr = st_mu_est(srt_exp = datax$spot_exp,  beta= beta.type)
exp_sc = sc_mu_est( sc_mu = ref_exp, beta = beta.type)

exp_rctd =  RCTDexp(beta = beta.type,  srt_exp = datax$spot_exp, ref_mu = ref_exp )

exp_spotdecon = spotdecon_est(srt_exp = datax$spot_exp,  beta= beta.type)


In [23]:
# Check and save exp_ENIGMA result
if (!file.exists(paste0("./results/MPOA/True_exp_ENIGMA_",patch_size,"um2.RDS")) ){
  exp_ENIGMA <- ENIGMA_est(
    spot_exp = datax_count$spot_exp, 
    sc_exp = datax_count$sc_exp, 
    sc_label = datax_count$sc_label, 
    beta = beta.type )
  
  # Save the result if computation is performed
  saveRDS(exp_ENIGMA, file = paste0("./results/MPOA/True_exp_ENIGMA_",patch_size,"um2.RDS"))
} else {
  # Load the result if it already exists
  exp_ENIGMA <- readRDS(paste0("./results/MPOA/True_exp_ENIGMA_",patch_size,"um2.RDS"))
}


In [24]:

# Check and save exp_TCA result
if (!file.exists("./results/MPOA/True_exp_TCA.RDS")) {
  
 exp_TCA <- TCA_est(spot_exp  = datax_count$spot_exp, beta= beta.type)
  
  # Save the result if computation is performed
  saveRDS(exp_TCA, file = "./results/MPOA/True_exp_TCA.RDS")
} else {
  # Load the result if it already exists
  exp_TCA <- readRDS("./results/MPOA/True_exp_TCA.RDS")
}

In [25]:
est_est= list(STged = stged.est.true$V.hat,
              STged.reg1 = stged.est.reg1$V.hat,
              STged.reg2 =stged.est.reg2$V.hat,
              STged.reg3 =stged.est.reg3$V.hat,
              RCTD = exp_rctd,
              TCA = exp_TCA,
              ENIGMA = exp_ENIGMA,
              ref_mu = exp_sc,
              LSR = exp_lsr,
              Spotdecon = exp_spotdecon)

files = paste0("./results/MPOA/True_spot_decon_",patch_size,"um2.RDS")

saveRDS(est_est, file =files)

In [26]:
## cell type proportion predected from the proposed cell type deconvolution methods
res_decon_ct = readRDS(paste0("./realdata/MPOA/Endeprop/merfish_Results.Deconv_",patch_size,"um.RDS"))
names(res_decon_ct)


[1] "CARD"          "RCTD"          "SpatialDWLS"   "SPOTlight"    
[5] "Stereoscope"   "cell2location" "EnDecon"       "DWLS"         
[9] "SONAR"

In [27]:
beta.est = res_decon_ct$EnDecon
beta.type.RCTD = res_decon_ct$RCTD
## use the esimated cell type proprotion

In [28]:
## use the esimated cell type proprotion
beta.type = beta.est

In [29]:

start_time <- Sys.time()

if (!file.exists(paste0("./results/MPOA/EST_stged_",patch_size,"um2.RDS"))) {
  stged.est.true <- MUR.STged(
    srt_exp = datax$spot_exp, 
    ref_exp = ref_exp, 
    beta.type = beta.type, 
    W = L.mat$weight_adj, 
    lambda1 = NULL, 
    lambda2 = NULL, 
    cutoff = 0.05, 
    epsilon = 1e-5
  )
  
  saveRDS(stged.est.true, file = paste0("./results/MPOA/EST_stged_",patch_size,"um2.RDS"))
} else {
  
  stged.est.true <- readRDS(paste0("./results/MPOA/EST_stged_",patch_size,"um2.RDS"))
}

end_time <- Sys.time()


reg1 <- TRUE

if (reg1 && !file.exists(paste0("./results/MPOA/EST_stged_reg1_",patch_size,"um2.RDS")) ){
  stged.est.reg1 <- MUR.STged(
    srt_exp = datax$spot_exp, 
    ref_exp = ref_exp, 
    beta.type = beta.type, 
    W = L.mat$weight_adj, 
    lambda1 = NULL, 
    lambda2 = 0, 
    cutoff = 0.05, 
    epsilon = 1e-5
  )
  
  saveRDS(stged.est.reg1, file = paste0("./results/MPOA/EST_stged_reg1_",patch_size,"um2.RDS"))
} else {
  
  stged.est.reg1 <- readRDS(paste0("./results/MPOA/EST_stged_reg1_",patch_size,"um2.RDS"))
}


reg2 <- TRUE
if (reg2 && !file.exists(paste0("./results/MPOA/EST_stged_reg2_",patch_size,"um2.RDS"))) {
  stged.est.reg2 <- MUR.STged(
    srt_exp = datax$spot_exp, 
    ref_exp = ref_exp, 
    beta.type = beta.type, 
    W = L.mat$weight_adj, 
    lambda1 = 0, 
    lambda2 = NULL, 
    cutoff = 0.05, 
    epsilon = 1e-5
  )
  
  saveRDS(stged.est.reg2, file = paste0("./results/MPOA/EST_stged_reg2_",patch_size,"um2.RDS"))
} else {
  
  stged.est.reg2 <- readRDS(paste0("./results/MPOA/EST_stged_reg2_",patch_size,"um2.RDS"))
}

reg3 <- TRUE
if (reg3 && !file.exists(paste0("./results/MPOA/EST_stged_reg3_",patch_size,"um2.RDS"))) {
  stged.est.reg3 <- MUR.STged(
    srt_exp = datax$spot_exp, 
    ref_exp = ref_exp, 
    beta.type = beta.type, 
    W = L.mat$weight_adj, 
    lambda1 = 0, 
    lambda2 =0, 
    cutoff = 0.05, 
    epsilon = 1e-5
  )
  
  saveRDS(stged.est.reg3, file = paste0("./results/MPOA/EST_stged_reg3_",patch_size,"um2.RDS"))
} else {
  
  stged.est.reg3 <- readRDS(paste0("./results/MPOA/EST_stged_reg3_",patch_size,"um2.RDS"))
}

We will adpote a value for lambda 1 in our algorithm... 
Select value of lambda1 0.09145015 
tuning for lambda 2 in our algorithm... 
Select value of lambda2 0.1951821 
Run the main algorithm... 


We will adpote a value for lambda 1 in our algorithm... 
Select value of lambda1 0.09145015 
Select value of lambda2 0 
Run the main algorithm... 


Select value of lambda1 0 
tuning for lambda 2 in our algorithm... 
Select value of lambda2 0.1951821 
Run the main algorithm... 


Select value of lambda1 0 
Select value of lambda2 0 
Run the main algorithm... 


In [30]:
exp_lsr = st_mu_est(srt_exp = datax$spot_exp,  beta= beta.type)

exp_sc = sc_mu_est( sc_mu = ref_exp, beta = beta.type)

exp_rctd =  RCTDexp(beta = beta.type.RCTD,  srt_exp = datax$spot_exp, ref_mu = ref_exp )

exp_spotdecon = spotdecon_est(srt_exp = datax_count$spot_exp,  beta= beta.type)

In [31]:
# Check and save exp_ENIGMA result
if (!file.exists(paste0("./results/MPOA/EST_exp_ENIGMA_",patch_size,"um2.RDS")) ){
  exp_ENIGMA = ENIGMA_est(spot_exp  = datax_count$spot_exp, 
                          sc_exp = datax_count$sc_exp, sc_label=datax_count$sc_label, 
                          beta= NULL)
  
  # Save the result if computation is performed
  saveRDS(exp_ENIGMA, file = paste0("./results/MPOA/EST_exp_ENIGMA_",patch_size,"um2.RDS"))
} else {
  # Load the result if it already exists
  exp_ENIGMA <- readRDS(paste0("./results/MPOA/EST_exp_ENIGMA_",patch_size,"um2.RDS")) # <-- Fixed here
}


In [32]:
# Check and save exp_TCA result
if (!file.exists("./results/MPOA/EST_exp_TCA.RDS")) {

  exp_TCA <- TCA_est(spot_exp  = datax_count$spot_exp, beta= beta.type)
  # Save the result if computation is performed
  saveRDS(exp_TCA, file ="./results/MPOA/EST_exp_TCA.RDS")
} else {
  # Load the result if it already exists
  exp_TCA <- readRDS("./results/MPOA/EST_exp_TCA.RDS")
}


In [33]:
est_est= list(STged = stged.est.true$V.hat,
              STged.reg1 = stged.est.reg1$V.hat,
              STged.reg2 =stged.est.reg2$V.hat,
              STged.reg3 =stged.est.reg3$V.hat,
              RCTD = exp_rctd,
              TCA = exp_TCA,
              ENIGMA = exp_ENIGMA,
              ref_mu = exp_sc,
              LSR = exp_lsr,
              Spotdecon = exp_spotdecon)

files = paste0("./results/MPOA/EST_spot_decon_",patch_size,"um2.RDS")

saveRDS(est_est, file =files)

Sensitive analysis

In [34]:
# use the true cell type propotions

beta.type = cell_type_prop[colnames(datax$spot_exp),]

In [35]:

lambda_values1 <- c(0,0.001,0.01,lambda1_opt,1,5,10)
lambda_values2 <- c(0,0.001,0.01,lambda2_opt ,1,5,10)

# Generate all combinations of lambda1 and lambda2 using expand.grid
parameter_grid <- expand.grid(lambda1 = lambda_values1, lambda2 = lambda_values2)

sens_STged = matrix(list(), nrow(parameter_grid) ,1)
for(ii in 1:nrow(parameter_grid)){
  
  cat("Run the STged", "\n")
  
  start_time <- Sys.time()
  stged.est = MUR.STged(srt_exp = datax$spot_exp, ref_exp = ref_exp, 
                        beta.type = beta.type,   W = L.mat$weight_adj, 
                        lambda1 = parameter_grid[ii,1], lambda2 = parameter_grid[ii,2],
                        cutoff = 0.05, 
                        epsilon = 1e-5)

  end_time <- Sys.time()
  sens_STged[[ii]] = stged.est$V.hat
  cat("Run time of STged", end_time - start_time,"\n")

}

files = paste0("./results/MPOA/True_spot_decon_",patch_size,"um2_sens.RDS")
saveRDS(sens_STged, file =files)


saveRDS(parameter_grid, file = paste0("./results/MPOA/True_cell_exp_tuning_values", patch_size, "um2.RDS"))

Run the STged 
Select value of lambda1 0 
Select value of lambda2 0 
Run the main algorithm... 
Run time of STged 5.610385 
Run the STged 
Select value of lambda1 0.001 
Select value of lambda2 0 
Run the main algorithm... 
Run time of STged 32.5023 
Run the STged 
Select value of lambda1 0.01 
Select value of lambda2 0 
Run the main algorithm... 
Run time of STged 43.42583 
Run the STged 
Select value of lambda1 0.08262865 
Select value of lambda2 0 
Run the main algorithm... 
Run time of STged 24.75637 
Run the STged 
Select value of lambda1 1 
Select value of lambda2 0 
Run the main algorithm... 
Run time of STged 23.69294 
Run the STged 
Select value of lambda1 5 
Select value of lambda2 0 
Run the main algorithm... 
Run time of STged 23.70032 
Run the STged 
Select value of lambda1 10 
Select value of lambda2 0 
Run the main algorithm... 
Run time of STged 24.30976 
Run the STged 
Select value of lambda1 0 
Select value of lambda2 0.001 
Run the main algorithm... 
Run time of STge

Benchmark cell type proprotions

In [36]:
res_decon_ct = readRDS("./realdata/MPOA/Endeprop/merfish_Results.Deconv_50um.RDS")

In [37]:
methods = names(res_decon_ct)
bench_celltype_STged = matrix(list(), length(methods) ,1)

for(ii in 1: length(methods) ){
  
  cat("Run the STged", methods[ii], "\n")
  beta.type.bench = res_decon_ct[[ii]]
  start_time <- Sys.time()
  stged.est = MUR.STged(srt_exp = datax$spot_exp, ref_exp = ref_exp, 
                        beta.type = beta.type.bench,   W = L.mat$weight_adj, 
                        lambda1 = NULL, lambda2 = NULL, cutoff = 0.05, 
                        epsilon = 1e-5)
                    
  end_time <- Sys.time()
  bench_celltype_STged[[ii]] = stged.est$V.hat
  cat("Run time of STged", end_time - start_time,"\n") 
  
}


files = paste0("./results/MPOA/STged_est_spot_decon_tuning.RDS")

saveRDS(bench_celltype_STged, file =files)

Run the STged CARD 
We will adpote a value for lambda 1 in our algorithm... 
Select value of lambda1 0.09650154 
tuning for lambda 2 in our algorithm... 
Select value of lambda2 0.1951821 
Run the main algorithm... 
Run time of STged 6.096897 
Run the STged RCTD 
We will adpote a value for lambda 1 in our algorithm... 
Select value of lambda1 0.1066524 
tuning for lambda 2 in our algorithm... 
Select value of lambda2 0.1951821 
Run the main algorithm... 
Run time of STged 17.05305 
Run the STged SpatialDWLS 
We will adpote a value for lambda 1 in our algorithm... 
Select value of lambda1 0.09373271 
tuning for lambda 2 in our algorithm... 
Select value of lambda2 0.1951821 
Run the main algorithm... 
Run time of STged 5.268862 
Run the STged SPOTlight 
We will adpote a value for lambda 1 in our algorithm... 
Select value of lambda1 0.08020027 
tuning for lambda 2 in our algorithm... 
Select value of lambda2 0.1951821 
Run the main algorithm... 
Run time of STged 6.216938 
Run the STged